In [187]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import pickle
from glob import glob
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import time

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps")

In [188]:
print(device)

mps


# Dataset

In [148]:
class ArgoverseDataset(torch.utils.data.Dataset):
    """Dataset class for Argoverse"""
    
    def __init__(self, 
                 data_path,
                 sample_indices):
        super(ArgoverseDataset, self).__init__()
        
        self.data_path = data_path
        self.sample_indices = sample_indices
        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.sample_indices)

    def __getitem__(self, idx):
        # Load one scene
        pkl_path = self.pkl_list[self.sample_indices[idx]]
        with open(pkl_path, 'rb') as f:
            scene = pickle.load(f)
            
        # the index of agent to be predicted 
        pred_id = np.where(scene["track_id"] == scene['agent_id'])[0][0]
        
        inp = {}
        # input: p_in & v_in; output: p_out
        inp['p_in'] = scene['p_in'][pred_id]
        inp['v_in'] = scene['v_in'][pred_id]
        acc_in = []
        for i, v in enumerate(inp['v_in']):
            if i==0:
                continue
            acc_in.append((inp['v_in'][i]-inp['v_in'][i-1])/(2/19))
        inp['acc_in'] = np.array(acc_in)
        # inp = np.append(np.append(p_in, v_in), acc_in)
        # print(inp.shape)
        
        p_out = scene['p_out'][pred_id]
        # v_out = scene['v_out'][pred_id]/np.linalg.norm(scene['v_out'][pred_id], axis=0).flatten()
        # acc_out = []
        # for i, v in enumerate(v_out):
        #     if i==len(v_out) or i==0:
        #         continue
        #     acc_out.append((v_out[i]-v_out[i-1])/(3/30))
        # acc_out = np.array(acc_out).flatten()
        # out = np.append(np.append(p_out, v_out), acc_out)
        # print(out.shape)
        
        # Convert to float torch tensor
        return inp, p_out

In [149]:
# Grid/Random Search

In [150]:
batch_size = 512
in_dim = 112
out_dim = 30*2
hidden_dim = 128
num_layers = 3
learning_rate = 0.01
decay_rate = 0.95
num_epoch = 100

# Data Loader

In [151]:
train_path = "./train/train"

# total number of scenes
indices = np.arange(0, 205942)

# train-valid split
np.random.shuffle(indices)
train_indices = indices[:180000]
valid_indices = indices[180000:]

# define datasets
train_set = ArgoverseDataset(train_path, train_indices)
valid_set = ArgoverseDataset(train_path, valid_indices)

# create dataloaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=1, shuffle=True, num_workers=0)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=1, shuffle=False, num_workers=0)

In [152]:
# def model(inp, tgt):
#     time = 2/19
#     pred = []
#     next_acc_arr = []
#     next_vel_arr = []
#     next_position_arr = []
    
#     acc_delta = (inp['acc_in'][:,-2]-inp['acc_in'][:,-1])/(time)
#     next_acc = inp['acc_in'][:,-1]+(acc_delta)*(time)
#     next_acc_arr.append(inp['acc_in'][:,-1])
#     next_acc_arr.append(next_acc)
    
#     next_vel = inp['v_in'][:, -1]+inp['acc_in'][:,-1]*(time)
#     next_vel_arr.append(next_vel)
    
#     next_position = inp['p_in'][:,-1]+inp['v_in'][:,-1]*(time)
#     next_position_arr.append(next_position)
    
#     pred.append(next_position)

#     for i in range(29):
#         time = 3/30
#         acc_delta = (next_acc_arr[:][-2]-next_acc_arr[:][-1])/(time)        
#         next_acc = next_acc_arr[:][-1]+(acc_delta[:][-1])*(time)
#         next_vel = next_vel_arr[:][-1]+next_acc_arr[:][-1]*(time)
#         next_position = next_position_arr[:][-1]+next_vel_arr[:][-1]*(time)
        
#         next_vel_arr.append(next_vel)
#         next_acc_arr.append(next_acc)
#         next_position_arr.append(next_position)
#         pred.append(next_position)
#     print('---------')
#     print(pred[0])
#     print(tgt[0])
#     return pred

In [176]:
def model(inp):
    time = 2/19
    positions = inp['p_in']
    
    curr_position = positions[:,-1]
    prev_position = positions[:,-2]

    prev_prev_position = positions[:,-3]
    prev_prev_prev_position = positions[:,-4]
    
    curr_velocity = (curr_position-prev_position)/time
    prev_velocity = (prev_position-prev_prev_position)/time
    prev_prev_velocity = (prev_prev_position-prev_prev_prev_position)/time
    
    curr_acceleration = (curr_velocity-prev_velocity)/time
    prev_acceleration = (prev_velocity-prev_prev_velocity)/time
    
    curr_acc_change = (curr_acceleration-prev_acceleration)/time
    
    pred_acceleration = curr_acceleration+curr_acc_change*time
    pred_velocity = curr_velocity+curr_acceleration*time
    pred_position = curr_position+curr_velocity*time
    
    positions = np.append(positions[0], pred_position, axis=0)
    
    for i in range(29):
        # need to alter times to correspond to right ones
        if i==0:
            time = 3/30
            time_prev = time_prev_prev = 2/19
        elif i==1:
            time = time_prev = 3/30
            time_prev_prev = 2/19
        else:
            time = time_prev = time_prev_prev = 3/30
        curr_position = positions[:][-1]
        prev_position = positions[:][-2]
        prev_prev_position = positions[:][-3]
        prev_prev_prev_position = positions[:][-4]
        
        curr_velocity = (curr_position-prev_position)/time
        prev_velocity = (prev_position-prev_prev_position)/time_prev
        prev_prev_velocity = (prev_prev_position-prev_prev_prev_position)/time_prev_prev

        curr_acceleration = (curr_velocity-prev_velocity)/time
        prev_acceleration = (prev_velocity-prev_prev_velocity)/time_prev

        curr_acc_change = (curr_acceleration-prev_acceleration)/time

        pred_acceleration = curr_acceleration+curr_acc_change*time
        pred_velocity = curr_velocity+curr_acceleration*time
        pred_position = curr_position+curr_velocity*time
        positions = np.reshape(np.append(positions, pred_position), (-1, 2))
        

    # print('---------')
    # print(positions[0])
    # print(tgt[0])
    return positions[19:]

In [185]:
from sklearn.metrics import mean_squared_error as MSE
train_mse = []

for inp, tgt in train_loader:
    tgt = np.reshape(tgt, (30, 2))
    # pred = torch.cat(model(inp, tgt))
    pred=model(inp)
    loss = MSE(pred, tgt)
    train_mse.append(loss) 
train_mse = round(np.sqrt(np.mean(train_mse)), 5)
print(train_mse)

5.47798


In [186]:
valid_mse = []
preds = []
trues = []
for inp, tgt in valid_loader:
    loss = 0
    pred = model(inp)
    tgt = np.reshape(tgt, (30, 2))
    loss = MSE(pred, tgt)
    preds.append(pred)
    trues.append(tgt)
    valid_mse.append(loss.item())

preds = np.concatenate(preds, axis = 0)  
trues = np.concatenate(trues, axis = 0)  
valid_mse = round(np.sqrt(np.mean(valid_mse)), 5)
print(valid_mse)

5.47515


# Evaluation and Submission

In [180]:
test_path = "./val_in/val_in/"
test_pkl_list = glob(os.path.join(test_path, '*'))
test_pkl_list.sort()

test_preds = []
for idx in range(len(test_pkl_list)):
    with open(test_pkl_list[idx], 'rb') as f:
        scene = pickle.load(f)
        # the index of agent to be predicted 
        pred_id = np.where(scene["track_id"] == scene['agent_id'])[0][0]
        
        inp = {}
        # input: p_in & v_in; output: p_out
        inp['p_in'] = np.array([scene['p_in'][pred_id]])
        inp['v_in'] = torch.tensor(scene['v_in'][pred_id])
        acc_in = []
        for i, v in enumerate(inp['v_in']):
            if i==0:
                continue
            acc_in.append((inp['v_in'][i]-inp['v_in'][i-1])/(2/19))
        inp['acc_in'] = np.array(acc_in)
        preds = model(inp)
        
        test_preds.append(preds)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


# Generate Submission File

In [183]:
# # Submission Files
sample_sub = pd.read_csv('sample_submission.csv')

In [184]:
# Convert to int
predictions = np.concatenate(test_preds).reshape(len(test_preds), -1).astype(int)
sub_df = pd.DataFrame(np.c_[sample_sub["ID"], predictions], columns=[np.r_[["ID"], ["v" + str(i) for i in range(1, 61)]]])
sub_df.to_csv('test_submission.csv', index=None)